**Table of contents**<a id='toc0_'></a>    
- [Robot Arm Eye to Hand Estimate Object Pose](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Initial Setup for pybullet](#toc3_)    
- [Generating the Robot Arm](#toc4_)    
- [Defining the Functions to be Used](#toc5_)    
- [Generating Colored Objects](#toc6_)    
- [Generating the Camera](#toc7_)    
- [Setting Camera Parameters](#toc8_)    
- [Capturing Images](#toc9_)    
- [Estimating the Position of the Colored Object](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Robot Arm Eye to Hand Estimate Object Pose](#toc0_)

In this notebook, we will generate a 6-axis robot arm and introduce a method to estimate the position of a specified colored object using a "fixed camera".

(Note: Since we are estimating the position from a fixed camera, we will not control the arm.)

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the time elapsed per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 3.5
camera_yaw = 180.0 # deg
camera_pitch = -40 # deg
cameraTargetPosition = [0, 0.5, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, cameraTargetPosition)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[Generating the Robot Arm](#toc0_)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0.0]  # Set the initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple6d_arm_with_gripper.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[Defining the Functions to be Used](#toc0_)

In [4]:
import cv2
import numpy as np

def detect_color_obj_pose(target_rgb, rgb_img, depth_img):
    """
    Function to obtain the center position, depth, and orientation of the first detected colored object
    
    Parameters
    ----------
    target_rgb : list
        RGB of the color to be detected
    rgb_img : numpy.ndarray
        Camera image (RGB)
    depth_img : numpy.ndarray
        Camera image (Depth)

    Returns
    -------
    obj_pose : numpy.ndarray
        Position and orientation of the colored object (x, y, z, roll, pitch, yaw)
    """

    # Convert the camera image to HSV format
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)

    # Convert RGB to HSV
    target_hsv = cv2.cvtColor(np.uint8([[target_rgb]]), cv2.COLOR_RGB2HSV)[0][0]

    # Specify the range of the color to be detected
    lower = np.array([target_hsv[0]-10, 50, 50])
    upper = np.array([target_hsv[0]+10, 255, 255])

    # Extract only the specified color
    mask = cv2.inRange(hsv_img, lower, upper)

    # Extract contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the contour with the largest area
    max_area = 0
    max_area_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_area_contour = contour

    # If no contour is found
    if max_area_contour is None:
        return None
    
    # Get the center position of the contour
    M = cv2.moments(max_area_contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    # Get the depth at the center position of the contour
    depth = depth_img[cy, cx]

    pos = [cx, cy, depth]
    
    return pos

def Rx(theta):
    """
    Calculate the rotation matrix around the x-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the x-axis
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    Calculate the rotation matrix around the y-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the y-axis
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    Calculate the rotation matrix around the z-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the z-axis
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])


def Hx(theta):
    """
    Calculate the homogeneous transformation matrix around the x-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the x-axis
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    Calculate the homogeneous transformation matrix around the y-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the y-axis
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    Calculate the homogeneous transformation matrix around the z-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the z-axis
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    Calculate the homogeneous transformation matrix for translation

    Parameters
    ----------
    x : float
        Translation in the x direction
    y : float
        Translation in the y direction
    z : float
        Translation in the z direction

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix for translation
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])

# <a id='toc6_'></a>[Generating Colored Objects](#toc0_)

In [5]:
# You can change this to get different results (make sure it fits within the field of view of the simpleCamera) ####
color_box_pos = [-2.0, 1.5, 0.05] # Set the initial position (x, y, z) of the colored object
#################################################################################

color_box_id = pybullet.loadURDF("../urdf/simple_box.urdf", color_box_pos, pybullet.getQuaternionFromEuler([0.0, 0.0, 0.0]), globalScaling=0.1, useFixedBase=True)

# <a id='toc7_'></a>[Generating the Camera](#toc0_)

In [6]:
import math

# You can change this to get different results (make sure the colorBox fits within the field of view of the simpleCamera) ####
SIMPLE_CAMERA_X = 0.0
SIMPLE_CAMERA_Y = 0.0
SIMPLE_CAMERA_Z = 4.0
SIMPLE_CAMERA_ROLL = 0.0
SIMPLE_CAMERA_PITCH = 0.0
SIMPLE_CAMERA_YAW = 0.0
#################################################################################

SIMPLE_CAMERA_ROLL = math.radians(180.0 + SIMPLE_CAMERA_ROLL)
SIMPLE_CAMERA_PITCH = math.radians(0.0 + SIMPLE_CAMERA_PITCH)
SIMPLE_CAMERA_YAW = math.radians(0.0 + SIMPLE_CAMERA_YAW)
simple_camera_id = pybullet.loadURDF("../urdf/simple_camera.urdf", [SIMPLE_CAMERA_X, SIMPLE_CAMERA_Y, SIMPLE_CAMERA_Z], pybullet.getQuaternionFromEuler([SIMPLE_CAMERA_ROLL, SIMPLE_CAMERA_PITCH, SIMPLE_CAMERA_YAW]), useFixedBase=True)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc8_'></a>[Setting Camera Parameters](#toc0_)

In [7]:
# Camera settings
fov = 60
image_width = 224
image_height = 224
aspect = image_width / image_height
near = 0.05
far = 5
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# Calculate the focal length
fov_rad = np.deg2rad(fov)
f = (image_height / 2) / np.tan(fov_rad / 2)

# Camera intrinsic parameters
camera_matrix = np.array([[f, 0, image_width//2],
                         [0, f, image_height//2],
                         [0, 0, 1]], dtype=np.float32)

# Distortion coefficients (assuming no distortion here)
dist_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc9_'></a>[Capturing Images](#toc0_)


In [8]:
SIMPLE_CAMERA_LINK_IDX = 0
SIMPLE_CAMERA_TARGET_LINK_IDX = 1

# Get the position of the camera
camera_link_pose = pybullet.getLinkState(simple_camera_id, SIMPLE_CAMERA_LINK_IDX)[0] # Position of the camera link at the end of the arm
camera_target_link_pose = pybullet.getLinkState(simple_camera_id, SIMPLE_CAMERA_TARGET_LINK_IDX)[0] # Position of a virtual link set just in front of the camera link
camera_link_orientation = pybullet.getEulerFromQuaternion(pybullet.getLinkState(simple_camera_id, SIMPLE_CAMERA_LINK_IDX)[1]) # Orientation of the camera link at the end of the arm

# Rotate the up vector of the camera according to the camera's orientation
camera_up_vector = np.array([0, -1, 0]) # Default up vector of the camera
R = Rz(SIMPLE_CAMERA_YAW) @ Ry(SIMPLE_CAMERA_PITCH) @ Rx(SIMPLE_CAMERA_ROLL)
rotate_camera_up_vector = R @ camera_up_vector

# Compute the view matrix of the camera
view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]], cameraTargetPosition=[camera_target_link_pose[0], camera_target_link_pose[1], camera_target_link_pose[2]], cameraUpVector=[rotate_camera_up_vector[0], rotate_camera_up_vector[1], rotate_camera_up_vector[2]])

# Get the camera image
_, _, rgb_img, depth_img, _ = pybullet.getCameraImage(
    width=image_width,
    height=image_height,
    viewMatrix=view_matrix,
    projectionMatrix=projection_matrix,
    renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
)

# Get the position of the object of the specified color
detect_color_rgb = [0, 0, 255] # Specify the color to be detected (red in this case)
pos = detect_color_obj_pose(detect_color_rgb, rgb_img, depth_img)
pixel_x = pos[0] # x-coordinate (position on the image)
pixel_y = pos[1] # y-coordinate (position on the image)
normal_z = pos[2] # z-coordinate (depth normalized to 0-1)
z = far * near / (far - (far - near) * normal_z) # Convert depth normalized to 0-1 to distance [m]
x = (pixel_x - image_width // 2) * z / f # Convert pixelX to distance [m]
y = (pixel_y - image_height // 2) * z / f # Convert pixelY to distance [m]

# <a id='toc10_'></a>[Estimating the Position of the Colored Object](#toc0_)

In [9]:
# Calculate the position of the colored object in the world coordinate system
CAMERA_POS = np.array([SIMPLE_CAMERA_X, 
                      SIMPLE_CAMERA_Y, 
                      SIMPLE_CAMERA_Z])
color_obj_pos_camera = np.array([x, y, z]) # Position of the object in the camera coordinate system
color_obj_pos_world = CAMERA_POS + R @ color_obj_pos_camera # Position of the object in the world coordinate system

# Get the position of the box
box_pos, box_orn = pybullet.getBasePositionAndOrientation(color_box_id)
print("color box obj pose: ", box_pos[0], box_pos[1], box_pos[2])
print("estimate color obj pose", color_obj_pos_world[0], color_obj_pos_world[1], color_obj_pos_world[2])

# Draw the position of the object on the Pybullet screen
pybullet.addUserDebugLine(CAMERA_POS[:3], color_obj_pos_world[:3], lineColorRGB=[1, 0, 0], lineWidth=5)

# Draw the position of the box
pybullet.addUserDebugText(f"true box pose ({box_pos[0]:.3f}, {box_pos[1]:.3f}, {box_pos[2]:.3f})", [4.0, 0.5, 0], textColorRGB=[1, 0, 0], textSize=1.3)
pybullet.addUserDebugText(f"eye to hand, estimate box pose ({color_obj_pos_world[0]:.3f}, {color_obj_pos_world[1]:.3f}, {color_obj_pos_world[2]:.3f})", [4.0, 1.0, 0.0], textColorRGB=[1, 0, 0], textSize=1.3)

color box obj pose:  -2.0 1.5 0.05
estimate color obj pose -1.9903058737195025 1.5078074800905317 0.10001191576652735


2